# This notebook launches the LatissCWFSAlign SAL script,
### The script is built for the scriptQueue but can be run via this notebook

#### This script is used to both collimated and focus the telescope. The script itself is not used to slew to a target, however, it is included in this procedure as an optional cell

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

In [ ]:
# Derive and index for the script that is specific to your user
print(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
print(f'The generated index is {index}')

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
# make sure all remotes etc are running
await script.start_task

In [ ]:
# set the script to have a DEBUG log level
script.log.level = logging.DEBUG

### The script does not acquire nor slew to a target, so this can be done here

In [ ]:
# Print the available instrument filter and gratings
# this is useful both for slewing and for the configuration steps below
inst_setup = await script.latiss.get_available_instrument_setup()
print(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

In [ ]:
# uncomment the following line to slew
# await script.atcs.slew_object(target)

In [ ]:
# uncomment the following line to take a snapshot to verify the target is positioned as expected 
# await script.atcs.offset_xy(x=20, y=20)

In [ ]:
# uncomment the following line to apply a telescope offset (in arcsec)
# await script.atcs.offset_xy(x=20, y=20)

## Begin configuring the script

In [ ]:
configuration = yaml.safe_dump({"filter": 'FELH0600', 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook'})

In [ ]:
# Set script state to UNCONFIGURED
# the next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.
# Therefore, it is included here despite being a non-operation in the first instance.
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# Set these values to None to verify a re-reduction does not happen of the images
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

In [ ]:
# run script, assigning the same group ID to all the images taken in the acquisition
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)

run_data = script.cmd_run.DataType()
await script.arun()

## Focusing and collimation are now complete

In [ ]:
# If required, then stop the telescope from tracking, but then you will lose your acquisition
# await script.atcs.stop_tracking()

### Optional Display of donuts and centroids

In [ ]:
%matplotlib inline

In [ ]:
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title(f"intra visitID - {script.intra_visit_id}")
ax11.imshow(script.I1[0].image0)
ax11.contour(script.algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title(f"extra visitID - {script.extra_visit_id}")
ax12.imshow(script.I2[0].image0)
ax12.contour(script.algo.pMask) 